# SNAC 
### Spiking Neural Autoencoder for time series Compression

This is a main script to test out the idea of Spiking neural networks compression of time series of sensor reads before packing up everyithing in a nice library.

First things first lets load up some sensor readings from an Arduino IoT node!

In [5]:
import numpy as np
import csv, torch
import torch.nn as nn
import torch.nn.functional as F
import snntorch as snn
import snntorch.functional as SF
from snntorch import spikeplot as splt
import matplotlib.pyplot as plt

dtype = torch.float
# See of the right GPU is selected
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device_name = torch.cuda.get_device_name(device)

print("Torch device: " + device_name)

# Sensor parameters
sample_period = 1  # ms
recording_length = 1e4  # ms
n_samples = recording_length / sample_period

ArduinoSensorsCSV = open("ArduinoSensors.csv")
sensor_recording = csv.reader(ArduinoSensorsCSV)
sensor_data = np.array([dataread for dataread in sensor_recording])
# Separate the three sensor readings
acc = sensor_data[:3]  # Accelerometer
mag = sensor_data[3:6]  # Magnetometer
gyr = sensor_data[6:]  # Gyroscope

Torch device: AMD Radeon RX 7800 XT


#### Sim Parameters 

Let's define the Autoencoder parameters.

In [ ]:
# Network Architecture/Parameters
# Encoder
en_num_inputs = 3
en_num_hidden = 30
en_num_latent = 3
# Decoder
de_num_inputs = 3
de_num_hidden_1 = 30
de_num_hidden_2 = 30
de_num_output = 3

# Temporal Dynamics
hid_beta = np.linspace(0.01, 0.99, en_num_hidden)
# hid_beta = np.linspace(0.4,0.8,en_num_hidden)#best sofar test new
hid_beta = torch.tensor(hid_beta, dtype=dtype)
out_beta = 0.3
hid_thresh = 40
thresh = 0.01